In [1]:
import requests
import pandas as pd

#PRACTIONER CLASS
class Practitioner:
    root_url = 'https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/Practitioner/'
    patient_list = []
    
    
    #INITIALIZE OBJECT USING PRACTITIONER ID
    def __init__(self, _id):
        self._id = _id
        search_prac = self.root_url + self._id
        self.data = requests.get(url=search_prac).json()
        
    #GET IDENTIFIER OF PRACTITIONER BASED ON ID    
    def get_identifier(self):
        return self.data['identifier'][0]['value']
    
    #RETRIEVE NAME BASED ON PRAC ID
    def retrieve_prac_details(self):
        namelist = self.data.get('name')
        
        prac_prefix = namelist[0].get('prefix')[0] 
        prac_given = namelist[0].get('given')[0]  
        prac_family = namelist[0].get('family')
        
        prac_name = prac_prefix + ' ' + prac_given + ' ' + prac_family
        
        result = ''.join([i for i in prac_name if not i.isdigit()])
        return result
    
    #METHOD USED TO RETIRIEVE PATIENTS WITH A CHOLESTROL READING THAT HAS HAD AN ENCOUNTER WITH A PRACTITIONER
    def monitor_current_patients(self):
        identifier = self.data['identifier'][0]['system'] + "|" + self.data['identifier'][0]['value']
        iterator = EncounterListIterator(identifier) #USE IDENTIFIER TO ITERATE THROUGH ENCOUNTERS
        
        patient_data = iterator.next_page() #RETURNS UNIQUE LIST OF PATIENTS
        self.patient_list = patient_data #STORE LIST OF PATIENTS
        iterator.extract_chol_data(patient_data)#GET CHOLESTROL DATA FOR THE LIST OF PATIENTS
        iterator.extract_bp_data(patient_data)#GET BP DATA FOR THE LIST OF PATIENTS
        
        
    #RETURNS PATIENTS THAT HAVE CHOLESTROL READING BUT NO ENCOUNTER WITH PRACTITIONER    
    def view_all_chol_patients(self):
        iterator = PatientListIterator("DiagnosticReport") #ITERATE THROUGH PATIENTS WITH CHOLESTROL READING
        patient_data = iterator.next_page() 
        
        pat = PatientList(patient_data)
        pat.get_chol_data() #GET CHOLESTROL DATA FOR LIST OF PATIENTS
    
    #GPRACTITIONER CAN ADD A PATIENT TO MONITOR
    def add_patient(self, patient_id):
        #print('Method Entered')
        self.patient_list.append(patient_id)
        print(self.patient_list)
        patient_list = PatientList([patient_id])
        patient_list.get_chol_data()
        
        
    #GPRACTITIONER CAN STOP MONITORING A PATIENT
    def remove_patient(self, patient_id):
        #print('Method Entered')
        for i in self.patient_list:
            print(i)
            if i == patient_id:
                index = self.patient_list.index(i)
                self.patient_list.pop(index)
        patient_list = PatientList(self.patient_list)
        patient_list.get_chol_data()
        
        
        
    #EXTRA REQUIREMENT - SET THRESHOLD SO ALL VALUES ABOVE THAT VALUE ARE DISPLAYED IN RED
    def set_threshold(self, val):
        #test_data = {'ID': ['29163', '93991', '106582'],'Value': [199.14, 162.68, 176.54],'Issued': ['2019-07-29T05:18:24.019+10:00','2016-10-13T12:03:19.997+11:00','2018-05-26T18:09:32.026+10:00']}
        threshold = val
        display = TableView()
        table = display.display_with_threshold(threshold)
    
    #SHOW EXTRA PATIENT DETAILS
    def show_patient_details(self, patient_id):
        patient_details = PatientList(patient_id)
        details = patient_details.extract_patient_data()
        #Need to print(details)
        display = TableView(details)
        table = display.display_patient_info()
      
    
    #SET HOW OFTEN THE VALUES SHOULD BE UPDATED WITH THE LATEST
    def set_update_time(self, n):
        pass
    
    #SET SYS AND DYS THRESHOLD - DISPLAY VALUES ABOVE THAT IN PURPLE
    def set_high_sys_dys(self, sys, dys):
        identifier = self.data['identifier'][0]['system'] + "|" + self.data['identifier'][0]['value']
        iterator = EncounterListIterator(identifier) #USE IDENTIFIER TO ITERATE THROUGH ENCOUNTERS
        
        patient_data = iterator.next_page() #RETURNS UNIQUE LIST OF PATIENTS
        self.patient_list = patient_data #STORE LIST OF PATIENTS
        thresh = iterator.extract_bp_data(patient_data)#GET BP DATA FOR THE LIST OF PATIENTS
        display = TableView(thresh)
        table = display.display_bp_and_chol(sys,dys)
        
    #SELECT PATIENT THAT PRAC WANTS TO MONITOR SYS OVER TIME    
    def show_sys_details(self, patient_id):
        patient_details = PatientList(patient_id)
        details = patient_details.extract_patient_data()
        #Need to print(details)
        display = Line(details)
        table = display.display_sys_overtime()
      
    def get_averages(self):
        iterator = PatientListIterator("DiagnosticReport") #ITERATE THROUGH PATIENTS 
        pat = TabaccoSmokingStatus(patient_data)
        num = pat.observation() #GET NUMBER OF NONE SMOKERS
        display = TableView(num) #DISPLAY AVERAGES IN TABLE FORMAT
    
        
    
    
        
        
    
 
        

In [2]:
import requests
from datetime import datetime

#ITERATOR CLASS HOLDS METHODS USED TO FETCH CERTAIN DATA FROM FHIR SERVER
class Iterator():
    
    def __init__(self, search_url):
        self.search_url = search_url
        
    #ABSTRACT METHOD - ITERATES THROUGH PAGES
    def next_page(self):              
        raise NotImplementedError("Subclass must implement abstract method")
    
    #ABSTRACT METHOD - ONCE DONE, SEND DATA TO LISTS
    def extract_chol_data(self, data):
        raise NotImplementedError("Subclass must implement abstract method")
        
        #ABSTRACT METHOD - ONCE DONE, SEND DATA TO LISTS
    def extract_bp_data(self, data):
        raise NotImplementedError("Subclass must implement abstract method")
        
              

In [3]:
import requests
from datetime import datetime

#INHERITS METHODS FROM ITERATOR
#GETS DATA SPECIFIC TO PATIENTS IN GENERAL
class PatientListIterator(Iterator):
    patient_array = []
    unique_list = []
    def __init__(self, search_url):
        Iterator.__init__(self, search_url)
        
    #LOOPS THROUGH FHIR SERVER TO GET ALL PATIENTS BASED ON THE SEARCH QUERY
    def next_page(self):
        root_url = 'https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/' + self.search_url
        dReports = requests.get(url=root_url).json()
        count_patient = 0
        entry = dReports['entry']
        for i in range(len(entry)):
            
            results = entry[i]['resource']['result']

            # Check whether this observation is on chterol or not.
            chterol = False
            for result in results:
                if result['display'] == 'Total Cholesterol':
                    chterol = True

                # If this observation is on cholesterol value, then record the patient's id and issued date.
            if chterol == True:
                patient_id = entry[i]['resource']['subject']['reference'][len('Patient/'):]
                self.patient_array.append(patient_id)
                
            for x in self.patient_array: 

                if x not in self.unique_list: 
                    self.unique_list.append(x)
                
        return self.unique_list   
    
    #GET THE CHOLESTROL DATA RELATED TO THE PATIENTS
    def extract_chol_data(self, data):
        encount_list = EncounterList(data)
        encount_list.get_chol_data()
        #print('sent') 
        
    def extract_bp_data(self, data):
        encount_list = EncounterList(data)
        encount_list.get_bp_data()
        #print('sent') 
    

In [4]:
import requests
from datetime import datetime

#THIS CLASS FINDS ALL ENCOUNTERS OF A PATIENT AND PRACTITIONER AND RETURNS A LIST OF PATIENT ID'S
class EncounterListIterator(Iterator):
    unique_list = []
    def __init__(self, search_url):
        Iterator.__init__(self, search_url)
        
    
    def next_page(self):
        root_url = 'https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/'
        beg_url = 'https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/Encounter?participant.identifier='
        end_url = '&_include=Encounter.participant.individual&_include=Encounter.patient&_count=50'
        identifier = self.search_url
        encounters_url = beg_url + identifier + end_url
        
        next_page = True
        next_url = encounters_url
        count_page = 0
        count_patient = 0
        patient_list=[]
        

        while next_page == True:
            all_encouters_practitioner = requests.get(url=encounters_url).json()

            # As discussed before, The monash FHIR server return results in a series of pages. 
            # here we check and record the next page 
            next_page = False
            links = all_encouters_practitioner['link']
            for i in range(len(links)):
                link=links[i]
                if link['relation'] == 'next':
                    next_page = True
                    next_url = link['url']
                    count_page += 1

            all_encouter_data=all_encouters_practitioner['entry']

            for entry in all_encouter_data:
                item = entry['resource']
                patient = item['subject']['reference']

                    # let's get the patient id, which we need to search for the cholesterol value
                patient_id=patient.split('/')[1]

                findCholesUrl = root_url +"Observation?patient="+patient_id+"&code=2093-3&_sort=date&_count=13"
                patientChol = requests.get(url=findCholesUrl).json()
                try:
                    cholData=patientChol['entry']

                    patient_list.append(patient_id)
                    #print(patient_id)
                except:
                    continue
                        # no data       
            for x in patient_list: 

                if x not in self.unique_list: 
                    self.unique_list.append(x)

            #print(unique_list)
            next_page = False
            
        #UNIQUE LIST HOLDS THE ID OF PATIENTS ENCOUNTERED WITH SAID PRACTITIONER
        #print(self.unique_list) 
        return self.unique_list
         
        
    def extract_chol_data(self, data):
        #SEND PATIENT DATA TO ENCOUNTERLIST TO GET CHOLESTROL DATA
        patient_list = PatientList(data)
        patient_list.get_chol_data()
        #print('sent')
        
    def extract_bp_data(self, data):
        #SEND PATIENT DATA TO ENCOUNTERLIST TO GET CHOLESTROL DATA
        patient_list = PatientList(data)
        patient_list.get_bp_data()
        #print('sent')
        
    

In [5]:
import requests
from datetime import datetime
import pandas as pd

#THE DATA SENT FROM ITERATOR
#ABSTRACT CLASS
class AbstractPatientList():
    
    def __init__(self, data):
        self.data = data
    
    def get_chol_data(self):
        raise NotImplementedError("Subclass must implement abstract method")
    
    def get_bp_data(self):
        raise NotImplementedError("Subclass must implement abstract method")
        
    def update(self, n):
        raise NotImplementedError("Subclass must implement abstract method")
        
    def disp_data(self):
        raise NotImplementedError("Subclass must implement abstract method")

In [17]:
import requests
from datetime import datetime
import pandas as pd

#DATA RELATING TO THE ENCOUNTERS BETWEEN PATIENTS AND PRACTITIONERS
class EncounterList(AbstractPatientList):
    
    chol_data = {}
    bp_data = {}
    def __init__(self, data):
        AbstractPatientList.__init__(self, data)
    
    def get_chol_data(self):
        chol_reading = CholestrolReading('2093-3')
        chol_data = chol_reading.get_observation(self.data)
        #print(chol_encounter)
        
        #GET PATIENT INFO FROM ID AND CHOL DATA TO SEND TO VIEW TO DISPLAY
        patient_list = chol_encounter['ID']
        patient_names = []
        for i in patient_list:
            patient1 = 'https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/Patient/' + i
            data = requests.get(url=patient1).json()
            namelist = data.get('name')
            patient_prefix = namelist[0].get('prefix')[0] 
            patient_given = namelist[0].get('given')[0]  
            patient_family = namelist[0].get('family')

            patient_name = patient_prefix + ' ' + patient_given + ' ' + patient_family

            result = ''.join([i for i in patient_name if not i.isdigit()])
            
            patient_names.append(result)
            
        chol_encounter['Patient Name'] = patient_names
        #print(chol_encounter)
        #CHOL_ENCOUNTER HOLDS THE VALUES FOR PATIENT, PATIENT CHOLESTROL AND ISSUED
        #NEEDS TO BE SENT TO VIEW TO DISPLAY IN TABLEVIEW
        #print(patient_names)
        
        #DISPLAY IN BAR GRAPH
        display = Bar(chol_encounter)
        display.display_latest_chol_data()
    
    
    def update(self, n):
        pass
    
    def get_bp_data(self):
        bp_reading = BloodPressureReading('55284-4')
        bp_data = bp_reading.get_bp_observation(self.data)
        
        
        #GET PATIENT INFO FROM ID AND CHOL DATA TO SEND TO VIEW TO DISPLAY
        patient_list = chol_encounter['ID']
        patient_names = []
        for i in patient_list:
            patient1 = 'https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/Patient/' + i
            data = requests.get(url=patient1).json()
            namelist = data.get('name')
            patient_prefix = namelist[0].get('prefix')[0] 
            patient_given = namelist[0].get('given')[0]  
            patient_family = namelist[0].get('family')

            patient_name = patient_prefix + ' ' + patient_given + ' ' + patient_family

            result = ''.join([i for i in patient_name if not i.isdigit()])
            
            patient_names.append(result)
            
        chol_encounter['Patient Name'] = patient_names
        #print(chol_encounter)
        #CHOL_ENCOUNTER HOLDS THE VALUES FOR PATIENT, PATIENT CHOLESTROL AND ISSUED
        #NEEDS TO BE SENT TO VIEW TO DISPLAY IN TABLEVIEW
        #print(patient_names)
        
        #DISPLAY IN TABLE FORMAT
        #display = TableView(chol_encounter)
        #display.display_latest_chol_data()
    
    #DISPLAYS CHOL AND BP DATA IN TABLE FORMAT VIA THE ENCOUNTERS/MONITORED PATIENTS
    def disp_data(self):   
        #DISPLAY IN TABLE GRAPH
        #ADD TWO DICT TOGETHER
        data = chol_data + bp_data
        display = TableView(data)
        display.display_latest_chol_data()
    
        

In [18]:
import requests
from datetime import datetime

#DATA FROM ITERATOR ABOUT PATIENTS
class PatientList(AbstractPatientList):
    chol_data = {}
    bp_data = {}
    
    def __init__(self, data):
        AbstractPatientList.__init__(self, data)
    
    def get_chol_data(self):
        chol_reading = CholestrolReading('2093-3')
        chol_encounter = chol_reading.get_observation(self.data)
        #print(chol_encounter)
        
        #GET PATIENT INFO FROM ID AND CHOL DATA TO SEND TO VIEW TO DISPLAY
        patient_list = chol_encounter['ID']
        patient_names = []
        for i in patient_list:
            patient1 = 'https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/Patient/' + i
            data = requests.get(url=patient1).json()
            namelist = data.get('name')
            patient_prefix = namelist[0].get('prefix')[0] 
            patient_given = namelist[0].get('given')[0]  
            patient_family = namelist[0].get('family')

            patient_name = patient_prefix + ' ' + patient_given + ' ' + patient_family

            result = ''.join([i for i in patient_name if not i.isdigit()])
            
            patient_names.append(result)
            
        chol_encounter['Patient Name'] = patient_names
        #print(chol_encounter)
        #CHOL_ENCOUNTER HOLDS THE VALUES FOR PATIENT, PATIENT CHOLESTROL AND ISSUED
        #NEEDS TO BE SENT TO VIEW TO DISPLAY IN TABLEVIEW
        #print(patient_names)
        display = TableView(chol_encounter)
        display.display_latest_chol_data()
    
    #GET DATA RELATING TO A SPECIFIC PATIENT
    def extract_patient_data(self):
        
        _id = self.data
        patient1 = 'https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/Patient/' + _id
        data = requests.get(url=patient1).json()
        
        namelist = data.get('name')
        patient_prefix = namelist[0].get('prefix')[0] 
        patient_given = namelist[0].get('given')[0]  
        patient_family = namelist[0].get('family')

        patient_name = patient_prefix + ' ' + patient_given + ' ' + patient_family

        result = ''.join([i for i in patient_name if not i.isdigit()])
        
        
        patient_data = {'ID': [data.get('id')], 'Name': [result], 'Birthdate': [data.get('birthDate')], 'City': [data.get('address')[0].get('city')], 'State': [data.get('address')[0].get('state')], 'Country': [data.get('address')[0].get('country')]}
        return patient_data
    
    def update(self, n):
        pass
    
    #DISPLAYS CHOL AND BP DATA IN TABLE FORMAT FOR ALL PATIENTS WITH CHOL AND BP READINGS
    def disp_data(self):   
        #DISPLAY IN TABLE GRAPH
        #ADD TWO DICT TOGETHER
        data = chol_data + bp_data
        display = TableView(data)
        display.display_latest_chol_data()
    

In [8]:
import requests
from datetime import datetime

#ALLOWS MONITORING OF CERTAIN METRICS
#ABSTRACT CLASS

class Diagnosis():
    
    def __init__(self, code):
        self.code = code
    
    #GETS OBSERVATION OF PATIENT CHOLESTROL
    def get_observation(self, data):
        raise NotImplementedError("Subclass must implement abstract method")
        
    def get_diagnosis_details(self):
        raise NotImplementedError("Subclass must implement abstract method")
        
   
        

In [9]:
import requests
from datetime import datetime

#GETS CHOLESTROL READING FOR A SPECIFIC DATA SET OF PATIENTS
class CholestrolReading(Diagnosis):
    chol_vals = []
    chol_issued = []
    def __init__(self, code):
        Diagnosis.__init__(self, code)
        
    def get_observation(self, data):
        for i in data:
            chol_code = self.code
            #print(chol_code)
            search3_url= "https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/Observation?patient="+i+"&code=" + chol_code + "&_sort=date"
            data2 = requests.get(url=search3_url).json()
            entry=data2['entry']
            check = entry[::-1][0]
            self.chol_issued.append(check['resource']['issued'])
            self.chol_vals.append(check['resource']['valueQuantity']['value'])
        
        data = {'ID': data,'Value': self.chol_vals,'Issued':self.chol_issued }
        #print(data)
        return data
    
            
    def get_diagnosis_details(self):
        pass
        
    

In [ ]:
import requests
from datetime import datetime

#GETS CHOLESTROL READING FOR A SPECIFIC DATA SET OF PATIENTS
class TabaccoSmokingStatus(Diagnosis):
    never_smoker = 0
    
    def __init__(self, code):
        Diagnosis.__init__(self, code)
        
    def get_observation(self, data):
        for i in range(data):
            smoker_code = self.code
            search3_url= "https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/Observation?patient="+i+"&code=" + bp_code + "&_sort=date"
            data2 = requests.get(url=search3_url).json()
            item = data2[i]['resource']['valueCodeableConcept']['text']
            if item = 'Never smoker':
                pass
            else: 
                never_smoker+= 1
            
        return never_smoker
            
    def get_diagnosis_details(self):
        pass
        

In [10]:
import requests
from datetime import datetime

#GETS CHOLESTROL READING FOR A SPECIFIC DATA SET OF PATIENTS
class BloodPressureReading(Diagnosis):
    bp_sys = []
    bp_dys = []
    bp_issued = []
    def __init__(self, code):
        Diagnosis.__init__(self, code)
        
    def get_observation(self, data):
        for i in range(data):
            bp_code = self.code
            #print(chol_code)
            search3_url= "https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/Observation?patient="+i+"&code=" + bp_code + "&_sort=date"
            data2 = requests.get(url=search3_url).json()
            entry=data2['entry']
            check = entry[::-1][0]
            item = entry[i]['resource']
            #GET PATIENT SYS [0] OF VALUE QUANTITY
            systolicbp = item['component'][0]['valueQuantity']['value']
            #GET PATIENT SYS [1] OF VALUE QUANTITY
            diastolicbp = item['component'][1]['valueQuantity']['value']
            #JUST GET THE DATE, LEAVE OUT THE REST
            issued = item['issued'][:10]
            
            self.bp_sys.append(systolicbp)
            self.bp_dys.append(diastolicbp)
            self.bp_issued.append(issued)
        
        data = {'ID': data,'Systolic Value': self.bp_sys, 'Dystolic Value': self.bp_dys,'Issued':self.bp_issued }
        #print(data)
        return data
            
    def get_diagnosis_details(self):
        pass
        

In [11]:
import requests
from datetime import datetime
import pandas as pd

#DISPLAYS DATA IN AN ATTRACTIVE FORMAT
class View():
    
    def __init__(self, data):
        self.data = data
    
    def display_latest_chol_data(self):
        raise NotImplementedError("Subclass must implement abstract method")
        
    def display_patient_info(self):
        raise NotImplementedError("Subclass must implement abstract method")
        
    def display_with_threshold(self):
        raise NotImplementedError("Subclass must implement abstract method")
        
    

In [12]:
import pandas as pd

#DISPLAYS DATA FROM LISTS IN A TABLE VIEW
class TableView(View):
    import pandas as pd
    #NUMBER OF CHOL ABOVE AVE
    average = 0
    #NUMBER OF HIGH SYS ABOVE ENTERED VAL
    high_sys = 0
    #NUMBER OF HIGH SYS ABOVE ENTERED VAL
    high_dys = 0
    
    
    def __init__(self, data):
        View.__init__(self, data)
    
    def display_latest_chol_data(self):
        data_to_disp = self.data
        df = pd.DataFrame(data_to_disp)
        #print(df)
        
        #GET AVERAGE AND HIGHLIGHT THOSE ABOVE IT
        df.style.highlight_null().render().split('\n')[:10]
        df.style
        tot = 0
        count = 0
        for i in data_to_disp['Value']:
            tot += i
            count += 1
        self.average = tot / count
        #print (self.average)
        #print (tot)
        #print (count)
        
        #IF VALUE IS ABOVE AVERAGE, PRINT THAT VALUE RED
        def color_ave_red(val):
            
            color = 'red' if val >= self.average else 'black'
            return 'color: %s' % color
        
        
        display(df.style.applymap(color_ave_red, subset = 'Value')) 
    
    #DISPLAY PATIENT INFO IN TABLE FORMAT
    def display_patient_info(self):
        df = pd.DataFrame(self.data) 
        df.style
        display(df)
    
    #DISPLAY ATTRIBUTES DIFFERENTLY DEPENDING ON ENTERED VALUE
    def display_with_threshold(self,thresh):
        data_to_disp = self.data
        df = pd.DataFrame(data_to_disp)
        #print(df)
        
        #GET THRESHOLD AND HIGHLIGHT THOSE ABOVE IT
        df.style.highlight_null().render().split('\n')[:10]
        df.style
        
        #METHOD TO SEE IF VALUE IS ABOVE THRESHOLD
        def color_thresh(val):
            
            color = 'red' if val >= thresh else 'black'
            return 'color: %s' % color
        
            
        display(df.style.applymap(color_thresh, subset = 'Value'))
        
    def display_bp_and_chol(self,sys,dys):
        #DISPLAY CHOL AND BP DATA TOGETHER
        data_to_disp = self.data
        df = pd.DataFrame(data_to_disp)
        #print(df)
        
        #GET THRESHOLD OF SYS AND DYS AND HIGHLIGHT THOSE ABOVE IT (PURPLE)
        df.style.highlight_null().render().split('\n')[:10]
        df.style
        
        #CHECK TO SEE IF VALUE IS HIGHER THAN ENTERED
        for i in data_to_disp['Value']:
            if i>sys:
                high_sys += 1
            if i>dys:
                high_dys += 1
            else:
                pass
        
        #METHOD TO SET PURPLE TO SYS AND DYS ABOVE VALUES
        def color_sys_thresh(val):
            
            color = 'purple' if val >= sys else 'black'
            return 'color: %s' % color
        
        def color_dys_thresh(val):
            
            color = 'purple' if val >= dys else 'black'
            return 'color: %s' % color
        
        
        display(df.style.applymap(color_sys_thresh, color_dys_thresh, subset = 'Value'))
        
    def display_averages(self,never_smoked):
        #DISPLAY AVERAGE CHOL, HIGH SYS AND HIGH DYS AND NUMBER OF NEVER SMOKED IN TABLE
        data_to_disp = {'Number of people with high Cholestrol:' : self.average, 
                        'Number of people with high Systolic Blood Pressure:' : self.high_sys,
                        'Number of people with high Dystolic Blood Pressure:' : self.high_dys,
                        'Number of people that have never smoked:' : never_smoked} #NUMBER OF PATIENT THAT HAVE NEVER SMOKED
        
        df = pd.DataFrame(data_to_disp)
        #print(df)
        df.style.highlight_null().render().split('\n')[:10]
        df.style
        display(df.style)
        

    

In [13]:
class Graph(View):
    
    def __init__(self, data):
        View.__init__(self, data)
        
    def display_latest_chol_data(self):
        pass
        
    def display_patient_info(self):
        pass
        
    def display_with_threshold(self):
        pass
        
    

In [14]:
class Line(Graph):
    
    value=[]
    time=[]
    
    def __init__(self, data):
        Graph.__init__(self, data)
        
    #DISPLAY CHOL DATA OVER A PERIOD OF TIME IN LINE GRAPH
    #METHOD PASSED NOW BUT CAN BE ADDED TO SHOW CHOL DATA USING THIS FORMAT
    def display_latest_chol_data(self):
        pass
        
    def display_patient_info(self):
        pass
        
    def display_with_threshold(self):
        pass
    
    #REQUIREMENT5
    #DISPLAYS SYSTOLIC DATA VALUES OF PATIENT OVERTIME 
    #COMES FROM DIAGNOSTIC (BP READING - HIGHER THAN THRESHOLD)
    
    def display_sys_overtime(self):
        data_to_disp = this.data;
        
        #LOOP THROUGH DATA TO RETRIEVE SYSTOLIC VALUES
        for i in range(len(data_to_disp)):
            item = entry[i]['resource']
            systolicbp = item['component'][0]['valueQuantity']['value']
            date= item['effectiveDateTime']
            yaxis = date[:10]
            #ADD VALUES TO VALUE[] AND TIME[] LISTS
            this.value.append(systolicbp)
            this.time.append(yaxis)
            
        #GET PATIENT NAME 
        namelist = data_to_disp('name')
        patient_prefix = namelist[0].get('prefix')[0] 
        patient_given = namelist[0].get('given')[0]  
        patient_family = namelist[0].get('family')

        patient_name = patient_prefix + ' ' + patient_given + ' ' + patient_family

        result = ''.join([i for i in patient_name if not i.isdigit()])  
        
        #CONTRUCT LINE GRAPH 
        x = time
        y = value
        fig= plt.figure()
        axes= fig.add_axes([0,0,1,1])
        plt.plot(x, y)
        plt.xlabel('Issued')
        plt.ylabel('Systolic Reading')
        plt.title('Monitoring for: ' + result)
        plt.show()

In [15]:
import matplotlib.pyplot as plt
class Bar(Graph):
   
    #DISPLAYS DATA FROM LISTS IN A BAR GRAPH VIEW 
    def __init__(self, data):
        Graph.__init__(self, data)
    
    #DISPLAYS LATEST CHOL DATA IN BAR GRAPH
    def display_latest_chol_data(self):
        data_to_disp = self.data
        fig = plt.figure()
        ax = fig.add_axes([0,0,1,1])
        patient_name = data_to_disp.get('Patient Name')
        chol_values = data_to_dis.get('Value')
        ax.bar(patient_name,chol_values)
        plt.xlabel('Patient')
        plt.ylabel('Cholestrol Reading')
        plt.show()
        
    def display_patient_info(self):
        pass
        
    def display_with_threshold(self):
        pass

In [28]:
p = Practitioner('1381208')

In [19]:
prac_id = input('Please enter Practitioner ID:').upper()
p = Practitioner(prac_id)

print ("Cholostrol Readings of Patients Based on Encounters")
print ("***************************************************")
p.monitor_current_patients()

threshold = input('Please enter Cholestrol Threshold:').upper()
p.set_threshold(int(threshold))

patient_id = input('Enter the ID of the patient you would like to see more data of:').upper()
p.show_patient_details(patient_id)


Please enter Practitioner ID:1381208
Cholostrol Readings of Patients Based on Encounters
***************************************************


ConnectionError: HTTPSConnectionPool(host='fhir.monash.edu', port=443): Max retries exceeded with url: /hapi-fhir-jpaserver/fhir/Observation?patient=106582&code=2093-3&_sort=date&_count=13 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11a04d150>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [24]:
print ("Cholostrol Readings of All Patients")
print ("***************************************************")
p.view_all_chol_patients()

Cholostrol Readings of All Patients
***************************************************


,ID,Value,Issued,Patient Name
0,7002659,184.420000,2017-05-27T13:01:13.247+10:00,Mr. Danial Heaney
1,7002749,184.620000,2019-02-24T20:48:03.645+11:00,Mr. Danial Huel
2,7002899,190.560000,2019-01-05T17:58:23.352+11:00,Mr. Danial Kshlerin
3,8345468,181.810000,2018-08-07T08:58:53.294+10:00,Mr. Minh Krajcik
4,7002977,180.420000,2019-04-12T11:13:59.706+10:00,Mr. Danial Legros
5,7003090,187.270000,2018-06-09T15:39:48.751+10:00,Mr. Danial Powlowski
6,8345563,169.700000,2017-01-04T14:22:59.721+11:00,Mrs. Minh Kub
7,7003175,181.660000,2018-03-09T03:38:54.768+11:00,Mr. Danial Satterfield
8,8345631,165.860000,2017-09-10T08:58:22.606+10:00,Mr. Minh Mitchell
9,7003320,173.400000,2019-06-16T17:51:24.548+10:00,Mr. Danial Schroeder


In [29]:
id_add = input('Please enter a patient to add to Monitor:').upper()
p.add_patient(id_add)

id_remove = input('Please enter a patient to add to remove:').upper()
p.remove_patient(id_remove)

Please enter a patient to add to Monitor:7003090


,ID,Value,Issued,Patient Name
0,29163,199.140000,2019-07-29T05:18:24.019+10:00,Mr. Adan Rogahn
1,93991,162.680000,2016-10-13T12:03:19.997+11:00,Ms. Albert Franecki
2,106582,176.540000,2018-05-26T18:09:32.026+10:00,Mrs. Aleisha Adams
3,7003090,187.270000,2018-06-09T15:39:48.751+10:00,Mr. Danial Powlowski


Please enter a patient to add to remove:29163


,ID,Value,Issued,Patient Name
0,93991,162.680000,2016-10-13T12:03:19.997+11:00,Ms. Albert Franecki
1,106582,176.540000,2018-05-26T18:09:32.026+10:00,Mrs. Aleisha Adams
2,7003090,187.270000,2018-06-09T15:39:48.751+10:00,Mr. Danial Powlowski


In [ ]:
x = [2, 4, 6]
y = [1, 3, 5]
plt.plot(x, y)
plt.show()